In [1]:
# coding=utf-8
import numpy
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile

dict_name = "teenslang_appwords.csv"

In [44]:
def Text_from_html_object(text):
    start = text.find("<")
    finish = text.find(">")
    text = text[:start] + text[finish + 1:]
    check_for_tag = text.find('<', start)
    while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
    return text

def Text_from_html_arr(arr):
    answer_text = ''
    for object in arr:
        text = str(object)
        check_for_tag = text.find('<')
        while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
        answer_text += text
    return answer_text

def get_page_count(soup):
    list_of_pages = soup.find_all('a', class_='page_number')
    try:
        last_page = list_of_pages[-2]
    except IndexError:
        return 1
    last_number = Text_from_html_object(str(last_page))
    return int(last_number)

def take_rus_letters_numbers(soup):
    links = []
    list_of_links = soup.find_all('a', class_='alph_menu')
    for link in list_of_links:
        pos = int(str(link).find("%D0%")) + 4
        name = str(str(link)[pos:pos+2])
        if name != "81":
            links.append(name)
    return links

def make_save(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name', 'definition', 'type', 'group', 'examples', 'origin', 'hashtags', 'synonyms'))

def add_to_save(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            type = word['type']
            group = word['group']
            examples = word['examples']
            origin = word['origin']
            hashtags = word['hashtags']
            synonyms = word['synonyms']
            writer.writerow((word['name'], word['definition'], word['type'], word['group'], 
                             word['examples'], word['origin'], word['hashtags'], word['synonyms']))
            

    
# and then rename mycsv.out to mycsv.csv if you wish.

def get_words_from_Teenslang(from_i, to_i, file):
    make_save(file)
    for j in range(from_i, to_i + 1):
        driver.get('http://teenslang.su/content/%D0%'+ rus_letters_numbers[j]) # download site to the Selenium
        html = driver.page_source # saving site source
        soup = BeautifulSoup(html, "lxml")
        page_count = get_page_count(soup)
        print('Буква алфавита № ' + str(j + 1) + ': ')
        dict = []
        for i in range(page_count):
            if i > 0:
                driver.get('http://teenslang.su/?page=' + str(i) + '&content=%D0%'+ rus_letters_numbers[j] + '*') # load sites
                html = driver.page_source # saving site source
                soup = BeautifulSoup(html, "lxml")
            print('Парсинг %d из %d' % (i + 1, page_count))
            table = soup.find('div', class_='visible-xs visible-md visible-sm visible-lg col-lg-6')
            words = table.find_all('div', class_='issueword1')

            for word_row in words:
                word_name = Text_from_html_object(str(word_row.find('span', class_='cap')))
                word_type = Text_from_html_object(str(word_row.find('span', class_='sta')))
                word_group = Text_from_html_object(str(word_row.find('span', class_='sph')))
                word_def = Text_from_html_object(str(word_row.find('span', class_='mea')))
                word_example = Text_from_html_object(str(word_row.find('span', class_='text')))
                word_story = Text_from_html_arr(word_row.find_all('div', class_='category'))
                word_hash = Text_from_html_object(str(word_row.find('span', class_='src')))
                word_syn = Text_from_html_object(str(word_row.find('span', class_='syn')))
                dict.append({
                    'num': i,
                    'name': brush(word_name),
                    'definition': word_def,
                    'type': " " if word_type == "NonNone" else word_type,
                    'group': " " if word_group == "NonNone" else word_group,
                    'examples': " " if word_example == "NonNone" else word_example,
                    'origin': " " if word_story == "NonNone" else word_story,
                    'hashtags': " " if word_hash == "NonNone" else word_hash,
                    'synonyms': " " if word_syn == "NonNone" else word_syn 
                })
        add_to_save(dict, file)
    fulldict = pd.read_csv(file)
    fulldict.drop_duplicates(keep='first')
    print("!!!!!!!!!!!ВСЕ!!!!!!!!!!")
            
        
def get_rus_letters_numbers():
    driver.get("http://teenslang.su/content/%D0%90")
    html = driver.page_source # saving site source
    soup = BeautifulSoup(html, "lxml")
    rus_letters_numbers = take_rus_letters_numbers(soup)
    return rus_letters_numbers

def brush(x):
    return x.capitalize().strip()

def print_dict_sorted_by_value(dictionary):
    for (key, value) in sorted(dictionary.items(), key=lambda x: x[1], reverse=True):
        print(key, ": ", value, sep='')

In [36]:
driver = webdriver.PhantomJS()
global driver
rus_letters_numbers = get_rus_letters_numbers()

In [37]:
file = "test_dict.csv"
get_words_from_Teenslang(from_i=9, to_i=9, file=file)

Буква алфавита № 10: 
Парсинг 1 из 1
!!!!!!!!!!!ВСЕ!!!!!!!!!!


In [38]:
dict = pd.read_csv(file)
dict.head(5)

,name,definition,type,group,examples,origin,hashtags,synonyms
0,Йа,я.,местоимение,,Йа так устала! • Когда йа была маленькой дево...,#Падонки,,
1,Йа креведко,"я знаю олбанский язык, но не совсем понимаю, о...",,(каменты),,#Падонки,,"я чайник, прикидываюсь шлангом, шлангую, йа кр..."
2,Йааазь,"язь, название рыбы.","-я, м.",(интернет),Вот она! Вот она рыба моей мечты: здоровенный ...,#Интернет,"Из любительского видеоролика, который оказалс...","йаазь, йазь, язззь"
3,Йад,яд.,"-а, м.",(каменты),,#Падонки,,
4,Йазык,"язык, чаще всего имеется в виду олбанский язык.","-а, м.",(каменты),,#Падонки,,


In [2]:
true_dict = pd.read_csv(dict_name)
true_dict.head(5)

,name,definition,type,group,examples,hashtags,origin,synonyms
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,_
1,А то,да — выражения подтверждения.,союз и частица,NaN,— Ты знаешь такого чела...? — А то!!!,#Общие,NaN,да.
2,Абаля,вот это конфуз!.,NaN,NaN,NaN,#Общие,NaN,_
3,Абанамат,восклицание раздражения или удивления.,междометие,NaN,Абанамат!!! Холодильник сломался не вовремя. ...,#Общие,NaN,"Искажение, эвфемизм матероного выражения, из п..."
4,Абапер,программист на языке ABAP/4.,"-а, м.","(айтишники\n, прогеры)","У абаперов, как правило, все проекты — это кос...",#Айтишники,"от англ. ABAPer, имеющего ровно то же значение.",АБАПер


In [5]:
true_dict.sort_values('name', ascending=True)

,name,definition,type,group,examples,hashtags,origin,synonyms
4201,Ё-моё,возглас выражения эмоций.,междометие,NaN,"Ё-моё, а я то думал що ты мне друг. • Е-мое, ...",#Общие,NaN,"вау, бля, блин."
4241,Ё-моё,возглас выражения эмоций.,междометие,NaN,"Ё-моё, а я то думал що ты мне друг. • Е-мое, ...",#Общие,NaN,"вау, бля, блин."
4301,Ёж,"водка, смешанная с пивом.","-а, м.",NaN,"Нас притащили на шикарную военку, где мы на ск...",#Общие,NaN,ёрш.
4300,Ёж,"осколочно-фугасный снаряд, обладающий малым пр...","-а, м.","(геймеры, симуляторы)",— слышь! ты кв2 голдой зарядил? — нет ежами!!!...,#Геймеры,NaN,"фугас, шапка."
4303,Ёж твою кочерыжку,"возглас, выражающий повышенные эмоции.",предикатив,NaN,"Ёж твою кочерыжку, как я сразу не догадался?!",#Общие,"Происхождение понятно и ежу: созвучие, эвфемизм.",_
4310,Ёжик,EDGE — технология высокоскоростной пакетной пе...,"-а, м.",(айтишники),Скорость передачи на ёжике быстрее чем на жопо...,#Айтишники,Англ. EDGE — Enhanced Data for Global Evolution.,_
4309,Ёжик,субноутбук «Asus EeePC».,"-а, м",(айтишники),Да купил ежика 701. • Очень мне понравилась к...,#Айтишники #Железячники,NaN,eeepc.
4315,Ёк-макарёк,употребляется при возмущении.,междометие,NaN,Ёк-макарёк! Что ж я наделал!,#Общие,NaN,"ёлки-палки, блин."
4317,Ёкалэмэнэ,"возглас выражения эмоций, интеллигентный вариа...","междометие, эвфемизм",NaN,"Екалэмэнэ, вот он и явился! • Екалэмэнэ, шо з...",#Общие,NaN,"ёклмн, еклмн опрст"
4318,Ёкарный Бабай,"непроизвольное восклицание человека, попавшего...",междометие,(морск. боцманск.),"Ёкарный Бабай, как вы мне все надоели!!!",#Общие,NaN,"кузькина мать, леший, чорт, Goddamn !!!"


In [73]:
true_dict = pd.read_csv(dict_name)


def replace_NonNone(cell):
    return "" if cell == "NonNone" else cell
 
    
def replace_NonNone_with_(cell):
    return "_" if cell == "NonNone" else cell
    '''new_array = []
    old = literal_eval(fulltext_array)
    for item in old:
        if item != "":
            new_array.append(item)
    return new_array'''

In [76]:
for column in true_dict:
    if column == 'synonyms':
        true_dict[column] = true_dict[column].apply(replace_NonNone_with_)
    true_dict[column] = true_dict[column].apply(replace_NonNone)


In [77]:
true_dict.head(5)

,name,definition,type,group,examples,hashtags,origin,synonyms
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,_
1,А то,да — выражения подтверждения.,союз и частица,,— Ты знаешь такого чела...? — А то!!!,#Общие,,да.
2,Абаля,вот это конфуз!.,,,,#Общие,,_
3,Абанамат,восклицание раздражения или удивления.,междометие,,Абанамат!!! Холодильник сломался не вовремя. ...,#Общие,,"Искажение, эвфемизм матероного выражения, из п..."
4,Абапер,программист на языке ABAP/4.,"-а, м.","(айтишники\n, прогеры)","У абаперов, как правило, все проекты — это кос...",#Айтишники,"от англ. ABAPer, имеющего ровно то же значение.",АБАПер


In [78]:
true_dict.to_csv(dict_name[:-4] + str(1) + ".csv", index=False)